In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will create a book recommendation algorithm using **K-Nearest Neighbors**.

You will use the [Book-Crossings dataset](http://www2.informatik.uni-freiburg.de/~cziegler/BX/). This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users. 

After importing and cleaning the data, use `NearestNeighbors` from `sklearn.neighbors` to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures distance to determine the “closeness” of instances.

Create a function named `get_recommends` that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

This code:

`get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")`

should return:

```
[
  'The Queen of the Damned (Vampire Chronicles (Paperback))',
  [
    ['Catch 22', 0.793983519077301], 
    ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479], 
    ['Interview with the Vampire', 0.7345068454742432],
    ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5376338362693787],
    ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178412199020386]
  ]
]
```

Notice that the data returned from `get_recommends()` is a list. The first element in the list is the book title passed in to the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed in to the function.

If you graph the dataset (optional), you will notice that most books are not rated frequently. To ensure statistical significance, remove from the dataset users with less than 200 ratings and books with less than 100 ratings.

The first three cells import libraries you may need and the data to use. The final cell is for testing. Write all your code in between those cells.

In [ ]:
[
 "Where the Heart Is (Oprah's Book Club (Paperback))",
 
 ["I'll Be Seeing You", 0.8016211],
 ['The Weight of Water', 0.77085835],
 ['The Surgeon', 0.7699411],
 ['I Know This Much Is True', 0.7677075]]

In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2021-01-10 17:33:23--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip      [   <=>              ]  24.88M  43.9MB/s    in 0.6s    

2021-01-10 17:33:24 (43.9 MB/s) - ‘book-crossings.zip’ saved [26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
df_ratings_over100books = df_ratings.groupby(["isbn"]).agg("count")

In [6]:
df_ratings_over100books = df_ratings_over100books[(df_ratings_over100books["rating"] >= 100)]

In [7]:
df_ratings_over200rating = df_ratings.groupby(["user"]).agg("count")

In [8]:
df_ratings_over200rating = df_ratings_over200rating[(df_ratings_over200rating["rating"] >= 200)]

In [9]:
df_ratings_edited = df_ratings[(df_ratings["user"].isin(df_ratings_over200rating.index)) & (df_ratings["isbn"].isin(df_ratings_over100books.index))]

In [10]:
df_books_edited = df_books[df_books["isbn"].isin(df_ratings_edited.groupby(["isbn"]).agg("count").index)]

In [11]:
df = pd.merge(right=df_ratings_edited, left = df_books_edited, on="isbn")

In [12]:
df = df.drop_duplicates(['title','user'])
piv = df.pivot(index='title', columns='user', values='rating')
piv = piv.fillna(0)


In [105]:
titles = list(piv.index.values)
X = piv.values

titles = list(piv.index.values)
X = piv.values

  #m = np.mean(X)
  #X=X-m

  

nn = NearestNeighbors(metric="cosine",algorithm="brute", p=2)

nn.fit(X)

row = title_2_index("Where the Heart Is (Oprah's Book Club (Paperback))")

dist, inds = nn.kneighbors(np.reshape(row,[1,-1]),len(titles),True)

In [106]:
def title_2_index(title):
    ind = titles.index(title)
    return X[ind,:]

def index_2_title(ind):
    return titles[ind]

243
614
597
240


0.8016211
0.77085835
0.7699411
0.7677075


In [56]:
inds[0][:6]

array([654, 539, 240, 597, 614, 243])

In [108]:
index_2_title(243)

"I'll Be Seeing You"

In [ ]:
I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True
0.8, 0.77, 0.77, 0.77

In [155]:
names

["I'll Be Seeing You",
 'The Weight of Water',
 'The Surgeon',
 'I Know This Much Is True']

In [115]:
distances = []
names = []
counter = 0
for i in reversed(inds[0][:6]):
  if counter == 4:break
  #print(i)
  names.append(index_2_title(i))
  counter += 1

counter = 0
for i in reversed(dist[0][:6]):
  if counter == 4:break
  distances.append(i)
    
  
  counter += 1

In [156]:
distances

[0.8016211, 0.77085835, 0.7699411, 0.7677075]

In [151]:
  recommended_books=[]


In [160]:
results = []

In [130]:
for i in range(4):
  print(names[i])

I'll Be Seeing You
The Weight of Water
The Surgeon
I Know This Much Is True


In [161]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  
  
  results = []
  recommended_books=[]
  recommended_books.append(book)
  
  

  for i in range(4):
    results.append([names[i], distances[i]])

  recommended_books.append(results)

  return recommended_books

In [162]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["I'll Be Seeing You", 0.8016211],
  ['The Weight of Water', 0.77085835],
  ['The Surgeon', 0.7699411],
  ['I Know This Much Is True', 0.7677075]]]

Use the cell below to test your function. The `test_book_recommendation()` function will inform you if you passed the challenge or need to keep trying.

In [163]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You havn't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075]]]
You passed the challenge! 🎉🎉🎉🎉🎉
